# Assignment 2: SQL Queries and Python interface to PostgreSQL

In [70]:
import psycopg, os

print('Connecting to the PostgreSQL database...')
conn = psycopg.connect(
    host="localhost",
    port='5432',
    dbname="my_db",
    user="postgres",
    password="123")

Connecting to the PostgreSQL database...


In [71]:
# create a cursor
cur = conn.cursor()

In [50]:
# Display the PostgreSQL database server version
print('PostgreSQL database version:')
cur.execute('SELECT version()')
db_version = cur.fetchone()
print(db_version)

PostgreSQL database version:
('PostgreSQL 16.1 (Debian 16.1-1.pgdg120+1) on aarch64-unknown-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',)


### Create A Relational Database

In [54]:
# Create Locations Table
cur.execute("""
CREATE TABLE Locations (
    location_id SERIAL PRIMARY KEY,
    city VARCHAR(100) NOT NULL,
    state VARCHAR(100),
    country VARCHAR(100) NOT NULL
)
""")

# Create CEOs Table
cur.execute("""
CREATE TABLE CEOs (
    ceo_id SERIAL PRIMARY KEY,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    year_of_birth INT NOT NULL,
    place_of_birth_id INT,
    year_became_ceo INT NOT NULL,
    FOREIGN KEY (place_of_birth_id) REFERENCES Locations(location_id)
)
""")

# Create Companies Table
cur.execute("""
CREATE TABLE Companies (
    company_id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    headquarter_city_id INT,
    ceo_id INT,
    founding_year INT NOT NULL,
    FOREIGN KEY (headquarter_city_id) REFERENCES Locations(location_id),
    FOREIGN KEY (ceo_id) REFERENCES CEOs(ceo_id)
)
""")

# Commit the transaction
conn.commit()

### Populate the Database

In [58]:
# Insert Locations
createCmd = """ INSERT INTO Locations (City, State, Country) VALUES 
('Mobile', 'AL', 'USA'),
('Cupertino', 'CA', 'USA'),
('West Godavari', 'Andhra-Pradesh', 'India'),
('Armonk', 'NY', 'USA'),
('Scarsdale', 'NY', 'USA'),
('Seattle', 'WA', 'USA'),
('Royal Oak', 'MI', 'USA'),
('Detroit', 'MI', 'USA'),
('Kumamoto', NULL, 'Japan'),
('Tokyo', NULL, 'Japan'),
('Shanghai', NULL, 'China'),
('Hangzhou', 'Zhejiang', 'China'),
('Rennes', 'Illes-et-Villaine', 'France'),
('Paris', NULL, 'France'),
('Seoul', NULL, 'South Korea'),
('Suwon', 'Gyeonggi-do', 'South Korea'),
('Memphis', 'TN', 'USA'),
('Bentonville', 'Arkansas', 'USA');
 """

cur.execute(createCmd)
conn.commit()

In [59]:
# Insert CEOs
createCmd = """INSERT INTO CEOs (first_name, last_name, year_of_birth, place_of_birth_id, year_became_ceo) VALUES 
('Tim', 'Cook', 1960, 1, 2011),
('Arvind', 'Krishna', 1962, 3, 2020),
('Andy', 'Jassy', 1968, 5, 2021),
('Mary', 'Barra', 1961, 7, 2014),
('Kenichiro', 'Yoshida', 1959, 9, 2018),
('Daniel', 'Zhang', 1972, 11, 2015),
('François-Henri', 'Pinault', 1962, 13, 2005),
('Lee', 'Jae-yong', 1968, 15, 2022),
('Doug', 'McMillon', 1966, 17, 2014);
"""

cur.execute(createCmd)
conn.commit()

In [60]:
# Insert Companies
createCmd = """INSERT INTO Companies (name, headquarter_city_id, ceo_id, founding_year) VALUES 
('Apple, Inc.', 2, 1, 1976),
('IBM', 4, 2, 1911),
('Amazon', 6, 3, 1994),
('General Motors', 8, 4, 1908),
('Sony Corporation', 10, 5, 1946),
('Alibaba Group', 12, 6, 1999),
('Kering SA', 14, 7, 1963),
('Samsung Electronics', 16, 8, 1969),
('Walmart', 18, 9, 1962);
"""

cur.execute(createCmd)
conn.commit()

### What are the first names of the CEOs of the companies headquartered in South Korea?

In [77]:
query_1 = """
SELECT ceos.first_name
FROM CEOs
JOIN Companies ON ceos.ceo_id = companies.ceo_id
JOIN Locations ON companies.headquarter_city_id = locations.location_id
WHERE locations.country = 'South Korea';
"""
cur.execute(query_1)
ceos_in_south_korea = cur.fetchone()
print(ceos_in_south_korea[0])

Lee


### Which city hosts the headquarters of the company whose CEO has a last name Yoshida?

In [78]:
query_2 = """
SELECT locations.city
FROM Locations
JOIN Companies ON locations.location_id = companies.headquarter_city_id
JOIN CEOs ON companies.ceo_id = ceos.ceo_id
WHERE ceos.last_name = 'Yoshida';
"""
cur.execute(query_2)
city_for_yoshida = cur.fetchone()
print(city_for_yoshida[0])

Tokyo


### What are the founding years of the companies whose CEOs were born between 1960 and 1969?

In [76]:
query_3 = """
SELECT companies.founding_year
FROM Companies
JOIN CEOs ON companies.ceo_id = ceos.ceo_id
WHERE ceos.year_of_birth BETWEEN 1960 AND 1969;
"""
cur.execute(query_3)
founding_years = cur.fetchone()

while founding_years is not None:
    print(founding_years[0])
    founding_years = cur.fetchone()

1976
1911
1994
1908
1963
1969
1962


### How long has the CEO of the company founded in 1908 been serving in their position?

In [79]:
query_4 = """
SELECT 2024 - ceos.year_became_ceo AS serving_years
FROM Companies
JOIN CEOs ON companies.ceo_id = ceos.ceo_id
WHERE companies.founding_year = 1908;
"""
cur.execute(query_4)
serving_years = cur.fetchone()
print(serving_years[0])

10


### What is the age of the CEO of the company based in Hangzhou?

In [80]:
query_5 = """
SELECT 2024 - ceos.year_of_birth AS ceo_age
FROM CEOs 
JOIN Companies ON ceos.ceo_id = companies.ceo_id
JOIN Locations ON companies.headquarter_city_id = locations.location_id
WHERE locations.city = 'Hangzhou';
"""
cur.execute(query_5)
ceo_age = cur.fetchone()
print(ceo_age[0])

52


In [81]:
# close the communication with the PostgreSQL
cur.close()